# Problem fomulation

In this project, we have some posts some of them are fake and some are real on raddit and as we know fake news has caused many social problems due to the raise of social network and its role in different domains such as politics. we are going to predict if a specific reddit post is fake news or not, by looking at its title. The data is raw has data in form of text which need to be preproccessed and extract feature from which are the posts (input) and a labels which represents the probability that this post will be fake( output).
Data mining function required is prediction.




*   What could be the challenges? Coverting raw data into useful features ⚠
*   What is the impact? We will be able to prevent many social problems caused by fake news 🚑

-What is an ideal solution? ✌
The ideal model is svm using embeddeding with word-ngram but with very high time complexity while logistic is less performance but takes less time complexity

Note: 
The questions and answers and the optimal solution are at the end of the notebook

---



# Preparing tools 🎨

In [ ]:
#Preparing our tools
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Reading Files ✍

In [ ]:
#Reading our dataset
data = pd.read_csv("/xy_train.csv")  #Reading our train set 
data1 = pd.read_csv("/x_test.csv")   #Reading our test set
data #printing our train set

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
#drop row that has label 2

data = data[data.label != 2]

In [ ]:
# Saving our ids in idfor later use
id=data1.id
id

0            0
1            1
2            2
3            3
4            4
         ...  
59146    59146
59147    59147
59148    59148
59149    59149
59150    59150
Name: id, Length: 59151, dtype: int64

In [ ]:
# Dropping id as it doesnt give us any useful information
data.drop(columns=['id'],axis='columns', inplace=True)
data1.drop(columns=['id'],axis='columns', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Text cleaning and processing steps-


*   Remove punctuations
*   Convert text to tokens

*   Remove stopwords using NLTK corpus stopwords list to match
*   Apply stemming

*   Convert words to feature vectors














In [ ]:
# Preparing some tools for word preprocessing 
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()

from pathlib import Path

# some seeting for pandas and hvplot

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

In [ ]:
# Importing more tools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
#Choosing the steemmer language english
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english")) #stopwords


def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
# Apply stemming to convert tokens to their root form. This is a rule-based process of word form conversion where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary.
        
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words 
            
            ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Clean texts for train dataset
data["text_clean"] = data.loc[data["text"].str.len() > 20, "text"]
data["text_clean"] = data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,text,label,text_clean
0,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...
...,...,...,...
59995,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish sniper simo yh invas finland ussr color
59996,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took kansa man year later get back
59997,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,safe smoke marijuana pregnanc surpris answer
59998,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realiz everyon room knife except bc


In [ ]:
# # Clean texts for test data
data1["text_clean"] = data1.loc[data1["text"].str.len() > 0, "text"]
data1["text_clean"] = data1["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
data1

,text,text_clean
0,stargazer,stargaz
1,yeah,yeah
2,PD: Phoenix car thief gets instructions from YouTube video,pd phoenix car thief get instruct youtub video
3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility",trump accus iran one problem credibl
4,"""Believers"" - Hezbollah 2011",believ hezbollah
...,...,...
59146,Bicycle taxi drivers of New Delhi,bicycl taxi driver new delhi
59147,Trump blows up GOP's formula for winning House races,trump blow gop formula win hous race
59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised",napoleon return exil island elba march colouris
59149,Deep down he always wanted to be a ballet dancer,deep alway want ballet dancer


In [ ]:
# Drop when any of x missing
data = data[(data["text_clean"] != "") & (data["text_clean"] != "null")]

data = data.dropna(axis="index", ).reset_index(drop=True)



In [ ]:
data

,text,label,text_clean
0,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...
...,...,...,...
59753,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish sniper simo yh invas finland ussr color
59754,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took kansa man year later get back
59755,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,safe smoke marijuana pregnanc surpris answer
59756,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realiz everyon room knife except bc


In [ ]:
data1

,text,text_clean
0,stargazer,stargaz
1,yeah,yeah
2,PD: Phoenix car thief gets instructions from YouTube video,pd phoenix car thief get instruct youtub video
3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility",trump accus iran one problem credibl
4,"""Believers"" - Hezbollah 2011",believ hezbollah
...,...,...
59146,Bicycle taxi drivers of New Delhi,bicycl taxi driver new delhi
59147,Trump blows up GOP's formula for winning House races,trump blow gop formula win hous race
59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised",napoleon return exil island elba march colouris
59149,Deep down he always wanted to be a ballet dancer,deep alway want ballet dancer


For our classification task, we want to be able to recognize whether a post has is fake or real. We make use of the label that come alongside with all texts. *Naturally, zero represents real and 1 represents the existanse of fake posts . We need to predict the probabality of being fake

### Descriptive analysis

Even though we deal with texts, we should still use some descriptive analysis to get a better understanding of the data:

In [ ]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data["text_clean"]).split()).value_counts()
word_freq[1:40]
len(word_freq)

40373

In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,angriff,1
1,delusion,1
2,wane,1
3,undament,1
4,miku,1
5,hatsun,1
6,nfler,1
7,hicock,1
8,mccall,1
9,wahr,1


# Feature creation with TF-IDF
Because classification models cannot deal with text data directly, we need to convert our comments to a numeric representation. As mentioned before, there are several ways to achieve this. This article provides a concise overview. All methods have in common that they assign each unique word in a document a unique number. A vector of numbers is created in which each element represents a word. Logically, the length of the vector will equal the number of unique words. In the simplest form (bag of words), a sentence can be represented by such a vector by indicating the presence of a word using a 1 in the appropriate index representing the word. All elements standing for words not included in the sentence will be 0.
Frequency methods improve on this very basic approach. For many applications, TF-IDF (term frequency, inverse document frequency) is a good choice. In our case, the TF part summarizes how often a word appears in a comment in relation to all words. As was mentioned earlier, that is not always a sufficient indicator for a useful word as it might be overly general or be used inflationary in many comments. This is where the IDF part comes into play. It downscales words that are prevalent in many other comments. Consequently, words that are frequent in a comment and also specific to it (i.e. they are uncommon in other comments) will get a high weight. Unspecific words or those with a low overall frequency will get a low weight.
This is how we apply TF-IDF to our comments using scikit-learn:

In [ ]:
"""
Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
vectorizer = TfidfVectorizer(
    analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
)
vectorizer.fit(data["text_clean"])

TfidfVectorizer(max_df=0.3, min_df=10, ngram_range=(1, 2))

In [ ]:
# Vector representation of vocabulary
word_vector = pd.Series(vectorizer.vocabulary_).sample(5, random_state=1)
print(f"Unique word (ngram) vector extract:\n\n {word_vector}")

Unique word (ngram) vector extract:

 eli       2666
go far    3595
bamboo     650
wisdom    9837
pocket    6705
dtype: int64


In [ ]:
#Checking missing values
def missing_values(data1):
    missing=pd.DataFrame(data1.isnull().sum()/len(data1))*100
    missing.columns = ['missing_values(%)']
    missing['missing_values(numbers)'] = pd.DataFrame(data1.isnull().sum())
    return missing.sort_values(by='missing_values(%)', ascending=False)
missing_values(data1)

,missing_values(%),missing_values(numbers)
text,0.0,0
text_clean,0.0,0


# Modeling
To test the classification performance of our model, we will perform a cross validation. For that, we split our data into a training and a testing set. The former is used to train the model and the latter to evaluate its predictions:

In [ ]:
#Setting X train and y actual and X test

X_train = data["text_clean"]
Y_train = data["label"]
X_test = data1["text_clean"]

print(X_train.shape)
print(X_test.shape)

(59758,)
(59151,)


In [ ]:
# # transform each sentence to numeric vector with tf-idf value as elements
# X_train_vec = vectorizer.transform(X_train)
# #X_test_vec = vectorizer.transform(X_test)
# X_train_vec.get_shape()

In [ ]:
# # Compare original comment text with its numeric vector representation
# print(f"Original sentence:\n{X_train[3:4].values}\n")
# # Feature Matrix
# features = pd.DataFrame(
#     X_train_vec[3:4].toarray(), columns=vectorizer.get_feature_names()
# )
# nonempty_feat = features.loc[:, (features != 0).any(axis=0)]
# print(f"Vector representation of sentence:\n {nonempty_feat}")

For this five word sentence, the vector of length 4107 contains mostly zeros. However, the indices representing the used words / ngrams are non empty. They include the value that `TF-IDF` assigned to them. In this particular case, "arzt empfehl" (recommend doctor) has the largest weight meaning that it is relatively frequent in our sentence while not being very common in other sentences of our dataset.

Now, that we have prepared our features, we can start to train and evaluate models. For a binary classification task there are many options to [chose from in `scikit-learn`](https://scikit-learn.org/stable/supervised_learning.html). We will focus on the ones that are most promising. In my experience they are: Logistic Regression, Support Vector Classification (SVC), Ensemble Methods (Boosting, Random Forest) and Neural Networks (i.e. Multi Layer Perceptron or MLP in sklearn). We will compare these models and chose the most promising one:

In [ ]:
# models to test
classifiers = [
    LogisticRegression(solver="sag", random_state=1),
    LinearSVC(random_state=1),
    RandomForestClassifier(random_state=1),
    XGBClassifier(random_state=1),
    MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ),
]
# get names of the objects in list (too lazy for c&p...)
names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]
print(f"Classifiers to test: {names}")

Classifiers to test: ['LogisticRegression', 'LinearSVC', 'RandomForestClassifier', 'XGBClassifier', 'MLPClassifier']


### Parameter tuning
WE will try tuning different models to find the optimal midel with the best parameterd
We've learned that linear SVC is a solid model choice and delivers great results out of the box. Still, we might be able to improve upon those by taking a more guided approach to choosing parameters. To do so, we will compare different parameters for feature creation as well as modeling. We can achieve this by making use of the pipeline and grid search functionality in sci-kit learn and random search

# Logistic Regression
In statistics, the logistic model is used to model the probability of a certain class or event existing such as pass/fail, win/lose, alive/dead or healthy/sick. This can be extended to model several classes of events such as determining whether an image contains a cat, dog, lion, etc
It is used in statistical software to understand the relationship between the dependent variable and one or more independent variables by estimating probabilities using a logistic regression equation. This type of analysis can help you predict the likelihood of an event happening or a choice being made.

In [ ]:
# feature creation and modelling in a single function
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("log", LogisticRegression())])
                 

# define parameter space to test # runtime 3hrs
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
   
}
# it is quite slow so we do 4 for now
pipe_clf =GridSearchCV(
    pipe, params, n_jobs=2, scoring="roc_auc", cv=4)
pipe_clf.fit(X_train, Y_train)


C:\Users\Dell\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('svc', LogisticRegression())]),
             n_jobs=2,
             param_grid={'tfidf__max_df': array([0.3]),
                         'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                         'tfidf__ngram_range': [(1, 2), (1, 3)]},
             scoring='roc_auc')

In [ ]:
print(pipe_clf.best_params_)

{'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2)}


In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] = pipe_clf.predict_proba(data1['text_clean'])[:,1]

submission.to_csv('Log.csv', index=False)


Score: 0.83893
Public score: 0.83739

# XGBoost classifier

In [ ]:
from sklearn.svm import SVC
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("XGC", XGBClassifier())])
                 

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
     
    # preprocessor__num__imputer__strategy points to preprocessor->num (a Pipeline)-> imputer -> strategy
    'XGC__n_estimators': [200, 300, 400],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'XGC__max_depth':[8,9,10,11]     
    
}
# it is quite slow so we do 4 for now
pipe_clf = RandomizedSearchCV(
    pipe, params, n_jobs=2, scoring="roc_auc", n_iter=3)
pipe_clf.fit(X_train, Y_train)
pickle.dump(pipe_clf, open("./clf_pipe.pck", "wb"))

C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:02:27] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
print(pipe_clf.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 30, 'tfidf__max_df': 0.3, 'XGC__n_estimators': 400, 'XGC__max_depth': 8}


In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] = pipe_clf.predict_proba(X_test)[:,1]

submission.to_csv('LOG_.csv', index=False)


0.71131 
### ☹



#### *From our observation we can find out that grid search is not the right choice for hypertuning our parameters as we have a very large data set which will require more computational time and power so random search will be more suitable and as well ysing a predifined split and compare between different models*

# Predefined split

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import train_test_split, RandomizedSearchCV


# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, Y_train, train_size = 0.7, stratify = Y_train, random_state = 3800)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X_train.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)





In [ ]:
# Creating a pipline for logistics regression
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("svc", LogisticRegression())])
                 

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    #  'svc__n_estimators': [20, 30, 40],  
    #  my_classifier__n_estimators points to my_classifier->n_estimators 
    'svc__penalty':['none', 'l1', 'l2', 'elasticnet']  ,    
    'svc__solver' :['newton-cg', 'lbfgs', 'liblinear'],
    'svc__C' :  [1e-5, 100]
}
pipe_clf1 =RandomizedSearchCV(
    pipe, params, n_jobs=2, scoring="roc_auc", cv=pds, n_iter=4)
pipe_clf1.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1 fits failed out of a total of 4.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('svc', LogisticRegression())]),
                   n_iter=4, n_jobs=2,
                   param_distributions={'svc__C': [1e-05, 100],
                                        'svc__penalty': ['none', 'l1', 'l2',
                                                         'elasticnet'],
                                        'svc__solver': ['newton-cg', 'lbfgs',
                                                        'liblinear'],
                                        'tfidf__max_df': array([0.3]),
                                        'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 

In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] = pipe_clf1.predict_proba(X_test)[:,1]

submission.to_csv('LOG1_.csv', index=False)

Score: 0.81254
Public score: 0.81070

#### *From my observations, I have found that Logistic Regression would be very effective on text data and the underlying algorithm is also fairly easy to understand. More importantly, in the NLP world, it's generally accepted that Logistic Regression is a great starter algorithm for text related classification and reduire less computaional time which makes it more effective.*

In [ ]:
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("XGC", XGBClassifier())])
                 

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
     
    # preprocessor__num__imputer__strategy points to preprocessor->num (a Pipeline)-> imputer -> strategy
    'XGC__n_estimators': [200, 300, 400],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'XGC__max_depth':[8,9,10,11]     
    
}
pipe_clf2 =RandomizedSearchCV(
    pipe, params, n_jobs=2, scoring="roc_auc", cv=pds, n_iter=4)
pipe_clf2.fit(X_train, Y_train)

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('XGC', XGBClassifier())]),
                   n_iter=4, n_jobs=2,
                   param_distributions={'XGC__max_depth': [8, 9, 10, 11],
                                        'XGC__n_estimators': [200, 300, 400],
                                        'tfidf__max_df': array([0.3]),
                                        'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                              

In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] = pipe_clf2.predict_proba(X_test)[:,1]

submission.to_csv('XG1_.csv', index=False)

Score: 0.80544
Public score: 0.80636

#### *We can see that didn't perform well. We can conclud that XGBoost does not perform so well on sparse and unstructured data. A common thing often forgotten is that Gradient Boosting is very sensitive to outliers since every classifier is forced to fix the errors in the predecessor learners. The overall method is hardly scalable.*

# Linear spport vector machine

In [ ]:

from sklearn.svm import SVC

# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", SVC(class_weight='balanced', kernel='linear',probability=True,))])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 3)],
    "tfidf__max_df": [0.5],
    "tfidf__min_df": [5],
    "svc__C": np.arange(0.2, 1, 0.15),
}
pipe_svc_clf = RandomizedSearchCV(pipe, params, n_jobs=2, scoring="roc_auc",  cv=pds, n_iter=4)
pipe_svc_clf.fit(X_train, Y_train)
#3hr 20mins

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('svc',
                                              SVC(class_weight='balanced',
                                                  kernel='linear',
                                                  probability=True))]),
                   n_iter=4, n_jobs=2,
                   param_distributions={'svc__C': array([0.2 , 0.35, 0.5 , 0.65, 0.8 , 0.95]),
                                        'tfidf__max_df': [0.5],
                                        'tfidf__min_df': [5],
                                        'tfidf__ngram_range': [(1, 3)]},
                   scoring='roc_auc')

In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] = pipe_svc_clf.predict_proba(X_test)[:,1]

submission.to_csv('SVM1_.csv', index=False)

Score: 0.83737
Private score: 0.83891

We can see that SVM performs the best for nlp tasks now in the next step we will add some parameters to search for the best of them

# Char n-gram

In [ ]:
# from sklearn.svm import SVC

# # feature creation and modelling in a single function
# pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", SVC(class_weight='balanced', kernel='linear',probability=True,))])

# # define parameter space to test # runtime 19min
# params = {
#     "tfidf__ngram_range": [(1, 3)],
#     "tfidf__max_df": [0.5],
#     "tfidf__min_df": [5],
#     "tfidf__analyzer":["char"],
     
#     "svc__C": np.arange(0.2, 1, 0.15),
# }
# pipe_svc_clf = RandomizedSearchCV(pipe, params, n_jobs=2, scoring="roc_auc",  cv=pds, n_iter=4)
# pipe_svc_clf.fit(X_train, Y_train)
# very Long computational time 

In [ ]:
# submission = pd.DataFrame()

# submission['id'] = id

# submission['label'] = pipe_svc_clf.predict_proba(X_test)[:,1]

# submission.to_csv('SVM2_.csv', index=False)

In [ ]:
# feature creation and modelling in a single function
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("log", LogisticRegression())])
                 

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    "tfidf__analyzer":["char"],
       
 
}
# it is quite slow so we do 4 for now
pipe_clf =RandomizedSearchCV(pipe, params, n_jobs=2, scoring="roc_auc",  cv=pds, n_iter=4)
pipe_clf.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('log', LogisticRegression())]),
                   n_iter=4, n_jobs=2,
                   param_distributions={'tfidf__analyzer': ['char'],
                                        'tfidf__max_df': array([0.3]),
                                        'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'tfidf__ngram_range': [(1, 2), (1, 3)]},
                   scoring=

In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] = pipe_clf.predict_proba(X_test)[:,1]

submission.to_csv('log2_.csv', index=False)

Score: 0.78521
Public score: 0.78176


We can observe that char n-gram doesn't gives us the best performance so we will continue using word n-gram whic is the defult

# Different preprocessing using embedding

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
#Choosing the steemmer language english
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english")) #stopwords


def clean_text(text, for_embedding=True):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
# Apply stemming to convert tokens to their root form. This is a rule-based process of word form conversion where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary.
        
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words 
            
            ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Clean texts for train dataset
data["text_clean"] = data.loc[data["text"].str.len() > 20, "text"]
data["text_clean"] = data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=True) if isinstance(x, str) else x
)
data

,text,label,text_clean
0,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group of friends began to volunteer at homeless shelter after their neighbors protested . Seeing...
1,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,British Prime Minister Theresa May on Nerve Attack on Former Russian Spy The government has conc...
2,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,"In , Goodyear released kit that allows PS to be brought to heel . https youtube com watch ALXulk..."
3,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,"Happy Birthday , Bob Barker ! The Price Is Right Host on How He Like to Be Remembered As the man..."
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,Obama to Nation Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Johns...
...,...,...,...
59753,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,"Finish Sniper Simo yh during the invasion of Finland by the USSR , colorized"
59754,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,"Nigerian Prince Scam took from Kansas man years later , he getting it back"
59755,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,Is It Safe To Smoke Marijuana During Pregnancy ? You Be Surprised Of The Answer no
59756,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,Julius Caesar upon realizing that everyone in the room has knife except him bc


In [ ]:
# # Clean texts for test data
data1["text_clean"] = data1.loc[data1["text"].str.len() > 0, "text"]
data1["text_clean"] = data1["text_clean"].map(
    lambda x: clean_text(x, for_embedding=True) if isinstance(x, str) else x
)
data1

,text,text_clean
0,stargazer,stargazer
1,yeah,yeah
2,PD: Phoenix car thief gets instructions from YouTube video,PD Phoenix car thief gets instructions from YouTube video
3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility","As Trump Accuses Iran , He Has One Problem His Own Credibility"
4,"""Believers"" - Hezbollah 2011",Believers Hezbollah
...,...,...
59146,Bicycle taxi drivers of New Delhi,Bicycle taxi drivers of New Delhi
59147,Trump blows up GOP's formula for winning House races,Trump blows up GOP formula for winning House races
59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised","Napoleon returns from his exile on the island of Elba . March , Colourised"
59149,Deep down he always wanted to be a ballet dancer,Deep down he always wanted to be ballet dancer


In [ ]:
# Drop when any of x missing
data = data[(data["text_clean"] != "") & (data["text_clean"] != "null")]

data = data.dropna(axis="index", ).reset_index(drop=True)



In [ ]:
#Setting X train and y actual and X test

X_train = data["text_clean"]
Y_train = data["label"]
X_test = data1["text_clean"]

print(X_train.shape)
print(X_test.shape)

(59758,)
(59151,)


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import train_test_split, RandomizedSearchCV


# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, Y_train, train_size = 0.7, stratify = Y_train, random_state = 3800)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X_train.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

In [ ]:
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("LOG", LogisticRegression())])
                 

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    #  'svc__n_estimators': [20, 30, 40],  
    #  my_classifier__n_estimators points to my_classifier->n_estimators 
    # 'svc__penalty':['none', 'l1', 'l2', 'elasticnet']  ,    
    # 'svc__solver' :['newton-cg', 'lbfgs', 'liblinear'],
    # 'svc__C' :  [1e-5, 100]
}
pipe_clf5 =RandomizedSearchCV(
    pipe, params, n_jobs=2, scoring="roc_auc", cv=pds, n_iter=5)
pipe_clf5.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('LOG', LogisticRegression())]),
                   n_iter=5, n_jobs=2,
                   param_distributions={'tfidf__max_df': array([0.3]),
                                        'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'tfidf__ngram_range': [(1, 2), (1, 3)]},
                   scoring='roc_auc')

In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] =pipe_clf5.predict_proba(X_test)[:,1]

submission.to_csv('log5_.csv', index=False)


Score: 0.85219
Public score: 0.85469

From our observation we can see that embedding  gives us the optimal performance. Embeddings make it easier to do machine learning on large inputs like sparse vectors representing words. Ideally, an embedding captures some of the semantics of the input by placing semantically similar inputs close together in the embedding space.

Now we will try performing embedding on our optimal model which is SVM

In [ ]:
from sklearn.svm import SVC
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("svc", SVC(class_weight='balanced', kernel='linear',probability=True,))
                 ]
                )

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 3)],
    "tfidf__max_df": [0.5],
    "tfidf__min_df": [5],
    "svc__C": np.arange(0.2, 1, 0.15),
}
pipe_svc_clf6 = RandomizedSearchCV(pipe, params, n_jobs=2, scoring="roc_auc",  cv=pds, n_iter=4)
pipe_svc_clf6.fit(X_train, Y_train)

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('svc',
                                              SVC(class_weight='balanced',
                                                  kernel='linear',
                                                  probability=True))]),
                   n_iter=4, n_jobs=2,
                   param_distributions={'svc__C': array([0.2 , 0.35, 0.5 , 0.65, 0.8 , 0.95]),
                                        'tfidf__max_df': [0.5],
                                        'tfidf__min_df': [5],
                                        'tfidf__ngram_range': [(1, 3)]},
                   scoring='roc_auc')

In [ ]:
submission = pd.DataFrame()

submission['id'] = id

submission['label'] =pipe_svc_clf6.predict_proba(X_test)[:,1]

submission.to_csv('svm5_.csv', index=False)

Score: 0.86339
Public score: 0.86436

We can the this is the optimal solution that gives us the best performance

### Conclusion

In this project we applied the a workflow for natural language processing with the goal to predict. After applying a pre processing and cleaning strategy, we have used a frequency based method (`TF-IDF`) to transform input texts to a numeric representation. These text vectors are the features used as input in our classification models. In the next step, we applied different models to our classification task and compared the results using a meaningful prediction and compared their scores. We've learned that this rather simple process is able to already produce decent results. Following, we have performed parameter tuning to further improve our model performance. 
In the next post, we will improve upon the outcome of this process. For that, we will apply more advanced methods. For one, this will include hyprid models. Moreover, we will use different, advanced implementations of Neural Networks as our classification models.

# Optimal model ☑
SVM
using embeeding and word-ngram

# Discussion (Questions & Answers) 🧠❓


---







### 🌈 What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?
An N-gram means a sequence of N words. N-grams are like a sliding window that moves across the word - a continuous sequence of characters of the specified length. Character N-Grams splits each word based on characters but word N-Grams splits the text based on words. Out-of-vocabulary (OOV) are terms that are not part of the normal lexicon found in a natural language processing environment.
Word n-gram tends to suffer more from OOV (Out-Of-Vocabulary) issue because of the new words that is presented in test dataset and doesn't exist in train dataset


---



### 🌈 What is the difference between stop word removal and stemming? Are these techniques language-dependent?
Stop word elimination and stemming are commonly used method in indexing. Stop words are high frequency words that have little semantic weight and are thus unlikely to help the retrieval process. Stemming conflates morphological variants of words in its root or stem.
These techniques are language dependant

---


### 🌈 Is tokenization techniques language dependent? Why?
Tokenization is essentially splitting a phrase, sentence, paragraph, or an entire text document into smaller units, such as individual words or terms
it is language dependent as it should know the meaning of each single word to split it depend on language.

---


### 🌈 What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?
- The main difference between them is that TfidfVectorizer performs both term frequency and inverse document frequency for you, while using TfidfTransformer will require you to use the CountVectorizer class from Scikit-Learn to perform Term Frequency. 
-It isn't feasible to use all possible n-grams
- We select based on the problem an important parameter that needs explanation is the ngram_range. An ngram of one means that you look at each word separately. An ngram of two (or bigram) means that you take the preceding and following word into account as well. In sentiment Analysis, setting n-gram ranges that use bigrams or trigrams can make great difference in the accuracy of classification,as they can capture more complex expressions formed by the composition of morethan one word. 
---

# References
https://link.springer.com/content/pdf/10.1007/978-81-8489-203-1_31.pdf
https://en.wikipedia.org/wiki/N-gram#:~:text=Out%2Dof%2Dvocabulary%20words,-Main%20article%3A%20Statistical&text=An%20issue%20when%20using%20n,or%20database%20during%20its%20preparation.
https://blog.marketmuse.com/glossary/word-vectors-definition/
https://blog.marketmuse.com/glossary/out-of-vocabulary-oov-definition/#:~:text=Out%2Dof%2Dvocabulary%20OOV,mathematical%20equivalent%20of%20word%20meaning
https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-ngram-tokenizer.html
https://www.analyticsvidhya.com/blog/2021/09/what-are-n-grams-and-how-to-implement-them-in-python/
https://towardsdatascience.com/understanding-word-n-grams-and-n-gram-probability-in-natural-language-processing-9d9eef0fa058#:~:text=An%20N%2Dgram%20means%20a,grams%2C%20which%20is%20quite%20interesting.
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
https://github.com/Shubha23/Text-processing-NLP/blob/mas
ter/NLP%20-%20Text%20processing%20pipeline.ipynb
https://itechindia.co/blog/which-of-the-3-algorithms-models-should-you-choose-for-sentiment-analysis-2/
https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275

https://developers.google.com/machine-learning/crash-course/embeddings/video-lecture
https://gate.ac.uk/sale/nle-svm/svm-ie.pdf
https://monkeylearn.com/text-classification-support-vector-machines-svm/#:~:text=From%20Texts%20to%20Vectors,encode%20any%20kind%20of%20data.
https://neptune.ai/blog/xgboost-everything-you-need-to-know